In [1]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib
import os
import pymorphy2
import re
from copy import deepcopy
import operator
import numpy as np

RusLem = pymorphy2.MorphAnalyzer()

def bprint(l, sep = " "):
    print sep.join(l)

In [3]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
 

#get list of sentences from html pages
def prepare_html_pages(html_dir):
    html_files = os.listdir(html_dir)
    sentences = []
    for filename in html_files:
        text = "" 
        with open(html_dir + filename, "r") as f:
            for line in f:
                text += line
                
        visible_text = text_from_html(text)
        sentences += visible_text.strip().split(".")
    
    return sentences


#sentences_normaliation
def normalization(sentences):
    for i, sent in enumerate(sentences):
        tokens = re.findall('[\w]+', sent.strip().lower(), re.U)
        sentences[i] = " ".join([RusLem.parse(token)[0].normal_form for token in tokens])
    return sentences


#creating forward index {sent_id: list of word} and token_dictionary
def create_forward_index(sentences):
    forward_index = {}
    token_list = set()
    for sent_id, sent in enumerate(sentences):
        if 1 < len(sent.strip().split()) <= 40:
            forward_index[sent_id] = sent.strip().split()
            for token in sent.strip().split():
                token_list.add(token)
        
    return forward_index, list(token_list)


def calculate_idf(forward_index, token_list):
    cnt_sentences = len(forward_index)
    token_df = {token:0 for token in token_list}
    for tokens in forward_index.values():
        for token in tokens:
            token_df[token] += 1
    token_idf = {token:np.log(cnt_sentences / float(token_df[token])) for token in token_df}
    return token_idf

def vec_normalization(vector):
    return vector / (1.0 * np.sqrt(np.sum(vector**2)))

def cos_similarity(vec_1, vec_2):
    return np.sum(vec_1 * vec_2) / float(np.sqrt(np.sum(vec_1**2)) * np.sqrt(np.sum(vec_2**2)))
    
class Collection(object):
    def __init__(self, htmls_dir="./htmls/"):
        self.raw_sentences = prepare_html_pages(htmls_dir)
        print len(self.raw_sentences)
        self.norm_sentences = normalization(deepcopy(self.raw_sentences))
        print len(self.norm_sentences)
        self.forward_index, self.token_list = create_forward_index(self.norm_sentences)
        print len(self.forward_index)
        self.token_idf = calculate_idf(self.forward_index, self.token_list)
        
    def doc2vec(self, sentence, idf=False):
        """
        Create sentence normalized vector
        """
        vector = [sentence.count(token) for token in self.token_list]
#         print sum(vector)
        if idf:
            vector = [c * np.log(self.token_idf[self.token_list[i]])
                      for i, c in enumerate(vector)]
            
        return np.asarray(vector)
        
    def sents2vectors(self):
        self.simple_vectors = {sent_idx:vec_normalization(
                                        self.doc2vec(self.forward_index[sent_idx], idf=False)
                                        )
                               for sent_idx in self.forward_index.keys()}
        
        self.idf_vectors = {sent_idx:vec_normalization(
                                    self.doc2vec(self.forward_index[sent_idx], idf=True)
                                    )
                               for sent_idx in self.forward_index.keys()}
        
    def ranking(self, query):
        norm_query = normalization([query])[0]
        bprint([norm_query])
        query_vec_simple = self.doc2vec(norm_query.strip().split(), idf=False)
        query_vec_idf = self.doc2vec(norm_query.strip().split(), idf=True)

        rank_simple = {}
        rank_tfidf = {}
        for sent_idx in self.simple_vectors.keys():
            rank_simple[sent_idx] = cos_similarity(query_vec_simple,
                                                   self.simple_vectors[sent_idx])
            rank_tfidf[sent_idx] = cos_similarity(query_vec_idf,
                                                   self.idf_vectors[sent_idx])
        
        return rank_simple, rank_tfidf
            

def print_serp(rank, query, sent_cnt = 10):
    sorted_rank = sorted(rank.items(), key = operator.itemgetter(1), reverse=True)
    print "Query:",
    bprint([query])
    print "\n"
    
    for idx, pair in enumerate(sorted_rank):
#         print idx, pair
        if idx > sent_cnt:
            break
        print pair[1], "\t",
        bprint(collection.forward_index[pair[0]])
        print

In [4]:
collection = Collection()
collection.sents2vectors()

1169
1169
933


In [5]:
query_1 = u"В начале XX века немецкий учитель математики, сам того не желая, научил лошадь считать."
query_2 = u"Искусственный язык для фантастической вселенной «Звёздный путь» создал профессиональный лингвист"
query_3 = u"На территории России были обнаружены останки ниппонозаврa, амурозаврa,целурозаврa и другие виды динозавров."

In [6]:
rank_simple_1, rank_idf_1 = collection.ranking(query=query_1)
rank_simple_2, rank_idf_2 = collection.ranking(query=query_2)
rank_simple_3, rank_idf_3 = collection.ranking(query=query_3)

в начало xx век немецкий учитель математика сам тот не желать научить лошадь считать
искусственный язык для фантастический вселенная звёздный путь создать профессиональный лингвист
на территория россия быть обнаружить останки ниппонозаврa амурозаврa целурозаврa и другой вид динозавр


In [7]:
print_serp(rank_simple_1, query_1)

Query: В начале XX века немецкий учитель математики, сам того не желая, научил лошадь считать.


0.284747398726 	в начало век популярный в сша стать блюз и джаз который сохранять свой господство в музыка до появление рок н ролл в 1950 х год

0.283069258536 	содержание скрыть 1 основной событие 2 главный изобретение 3 использование сочетание xx век в название 4 двадцатый век в искусство 5 смотреть

0.283069258536 	также править править вика текст в викитека есть текст по тема документ xx век xx век хронология изобретение

0.283069258536 	xxii век править править вика текст в начало xxii век в клингонский общество повсеместно усилиться влияние класс воин

0.277350098113 	в многий страна в тот число и в россия в 21 век наблюдаться тенденция к снижение престижность педагогический профессия и как следствие недооценённость учительский труд

0.273861278753 	в сша один из крупный киностудия называться xx век фокс

0.258198889747 	учитель в образовательный процесс править править вика текст мон

In [8]:
print_serp(rank_idf_1, query_1)

Query: В начале XX века немецкий учитель математики, сам того не желая, научил лошадь считать.


0.30095035986 	ганс считать

0.242162149102 	другой слово ганс быть действительно феноменально умный лошадь и прекрасно понимать что от он хотеть но конечно ни математика ни немецкий язык он не знать и не понимать

0.217894923247 	также править править вика текст в викитека есть текст по тема документ xx век xx век хронология изобретение

0.185836111712 	комиссия возглавить философ и психолог карл штумпф в состав её войти самый разный человек чей профессия быть так или иначе связать с лошадь математика или психология врач ветеринар владелец цирк офицер кавалерия несколько школьный учитель математика и директор берлинский зоопарк

0.16384848594 	название популярный советский боевик пират xx век

0.151959371158 	начало эпидемия спид

0.149983079329 	в сша один из крупный киностудия называться xx век фокс

0.147979632671 	некоторый считать что дромеозаврид группа манираптор хороший классифицир

In [9]:
print_serp(rank_simple_2, query_2)

Query: Искусственный язык для фантастической вселенной «Звёздный путь» создал профессиональный лингвист


0.36514837167 	звёздный путь англ

0.363912671437 	он известный как сценарист другой сериал в вселенная звёздный путь звёздный путь глубокий космос 9 1993 1999 звёздный путь в яджер 1995 2001

0.350823207723 	вселенная звёздный путь один из наиболее детально проработать вымышленный вселенная 1

0.34749779421 	он являться режиссёр и сценарист три фильм из серия звёздный путь звёздный путь ii гнев хан 1982 звёздный путь iv путешествие домой 1986 звёздный путь vi неоткрытый страна 1991

0.338061701891 	основный статья звёздный путь энтерпрайза звёздный путь энтерпра йз англ

0.335410196625 	он создать специально для фильм лингвист марк окранд

0.316227766017 	2 звёздный путь 1

0.310086836473 	marc okrand mɑrk ˈoʊkrænd 3 июль 1948 американский лингвист известный как создатель язык для народ фантастический мир кинематограф сша клингонский вулканский и атлантский язык

0.282842712475 	з

In [10]:
print_serp(rank_idf_2, query_2)

Query: Искусственный язык для фантастической вселенной «Звёздный путь» создал профессиональный лингвист


0.349282738206 	он создать специально для фильм лингвист марк окранд

0.288200424825 	marc okrand mɑrk ˈoʊkrænd 3 июль 1948 американский лингвист известный как создатель язык для народ фантастический мир кинематограф сша клингонский вулканский и атлантский язык

0.243760095689 	вселенная звёздный путь один из наиболее детально проработать вымышленный вселенная 1

0.222160411704 	он известный как сценарист другой сериал в вселенная звёздный путь звёздный путь глубокий космос 9 1993 1999 звёздный путь в яджер 1995 2001

0.215633043521 	можно выделить следующий уровень профессиональный подготовка педагог существующий в настоящее время уровень профессиональный ориентация педагогический класс школа среднее профессиональный образование педагогический колледж высокий профессиональный образование высокий учебный заведение подготовка научно педагогический кадр для высокий профессиональный о

In [11]:
print_serp(rank_simple_3, query_3)

Query: На территории России были обнаружены останки ниппонозаврa, амурозаврa,целурозаврa и другие виды динозавров.


0.362738125055 	первый и единственный неполный скелет быть обнаружить японец в 1934 год на территория больница в синегорск сахалин

0.3 	уметь слушать и слышать другой и иной мнение

0.283980917124 	транспорт тысячелетие основать на конный тяга быть на протяжение хх век заменить на грузовой автомобиль и автобус что стать возможный благодаря крупномасштабный эксплуатация ископаемое топливо

0.282842712475 	скульптура учитель на территория мади

0.279751442472 	среди вопрос на который он давать ответ быть не только такой как сколько быть 12 12 но и например если восьмой день месяц приходиться на вторник то какой день по счёт быть следующий пятница

0.273861278753 	для клингонский важно быть быть непохожий на привычный земной язык

0.258198889747 	результат исследование пфунгст быть принять научный сообщество и использоваться в эксперимент по интеллект животное и человек чт

In [12]:
print_serp(rank_idf_3, query_3)

Query: На территории России были обнаружены останки ниппонозаврa, амурозаврa,целурозаврa и другие виды динозавров.


0.286927874807 	первый и единственный неполный скелет быть обнаружить японец в 1934 год на территория больница в синегорск сахалин

0.24572800686 	скульптура учитель на территория мади

0.21050651436 	единственный известный вид amurosaurus riabinini назвать в честь покойный палеонтолог анатолий рябинин который возглавить первый русский экспедиция 1916 и 1917 год для поиск окаменелый останки динозавр 4 5

0.159383066239 	помогать другой

0.149710852665 	орнитомимозавр немалоизвестный похожий на страус динозавр

0.148047065197 	пандемия а в конец век быть обнаружить новое вирусный заболевание спид который возникнуть в африка

0.147064411445 	amurosaurus родиться птицетазовый динозавр из подсемейство ламбеозаврин жить в конец меловой период верхний маастрихта 2 найти в россия в благовещенск 3

0.143146067901 	быть вскрыть лишь небольшой часть слой но 90 найти останки принад